In [3]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

In [4]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(url)

In [6]:
column_names = {'X1': 'Relative Compactness',
                'X2': 'Surface Area',
                'X3': 'Wall Area',
                'X4': 'Roof Area',
                'X5': 'Overall Height',
                'X6': 'Orientation',
                'X7': 'Glazing Area',
                'X8': 'Glazing Area Distribution',
                'Y1': 'Heating Load',
                'Y2': 'Cooling Load'
}
df = df.rename(columns = column_names)

In [8]:
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [10]:
features_df = normalized_df.drop(columns = ['Heating Load', 'Cooling Load'])
heating_target = normalized_df['Heating Load']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target, test_size = 0.3, random_state = 1)

In [12]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [14]:
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

In [15]:
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

### To compare the effects of regularization 

In [16]:
def get_weights_df(model, feature, col_name):
    #This function returns the weight of every feature
    weights = pd.Series(model.coef_, feature.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [17]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear Model Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso Weight')

In [23]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on = 'Features')

In [25]:
final_weights = pd.merge(final_weights, lasso_weights_df, on = 'Features')

In [26]:
final_weights

,Features,Linear Model Weight,Ridge Weight,Lasso Weight
0,Surface Area,-4.843400e+12,-0.062275,0.000000
1,Relative Compactness,-6.064125e-01,-0.283471,-0.027719
2,Orientation,1.937866e-03,0.003369,0.000000
3,Glazing Area Distribution,2.604699e-02,0.029088,0.021431
4,Glazing Area,2.170086e-01,0.212449,0.206132
5,Overall Height,3.916016e-01,0.442467,0.463482
6,Wall Area,2.825317e+12,0.103061,0.200087
7,Roof Area,3.632550e+12,-0.163192,-0.000000
